<a href="https://colab.research.google.com/github/rahiakela/natural-language-processing-case-studies/blob/master/huggingface-transformers-practice/transformers-use-cases/6_text_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Text Summarization

Summarization is the task of summarizing a document or an article into a shorter text. If you would like to fine-tune a model on a summarization task, you may leverage the [run_summarization.py script](https://github.com/huggingface/transformers/blob/master/examples/seq2seq/run_summarization.py).

An example of a summarization dataset is the CNN / Daily Mail dataset, which consists of long news articles and was created for the task of summarization. If you would like to fine-tune a model on a summarization task, various approaches are described in [this document](https://github.com/huggingface/transformers/blob/master/examples/seq2seq/README.md).

Referemce: https://huggingface.co/transformers/task_summary.html

## Setup

In [1]:
import tensorflow as tf
import torch

In [ ]:
!pip install transformers

In [3]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import TFAutoModelWithLMHead

from pprint import pprint

## Loading Pipeline

Here is an example of using the pipelines to do summarization. It leverages a Bart model that was fine-tuned on the CNN / Daily Mail data set.

In [ ]:
summarizer = pipeline("summarization")

ARTICLE = """New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
 A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
 Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
 In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
 Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
 2010 marriage license application, according to court documents.
 Prosecutors said the marriages were part of an immigration scam.
 On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
 After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
 Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
 All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
 Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
 Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
 The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
 Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
 Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
 If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

Because the summarization pipeline depends on the `PreTrainedModel.generate()` method, we can override the default arguments of `PreTrainedModel.generate()` directly in the pipeline for `max_length` and `min_length` as shown below. 

This outputs the following summary:

In [6]:
print(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False))

[{'summary_text': ' In total, she has been married to nine men since 1999 . She faces two counts of "offering a false instrument for filing in the first degree" Her next court appearance is May 18 .'}]


Here is an example of doing summarization using a model and a tokenizer. The process is the following:

1. Instantiate a tokenizer and a model from the checkpoint name. Summarization is usually done using an encoder-decoder model, such as `Bart` or `T5`.
2. Define the article that should be summarized.
3. Add the T5 specific prefix “summarize: “.
4. Use the PreTrainedModel.generate() method to generate the summary.

In this example we use Google`s T5 model. Even though it was pre-trained only on a multi-task mixed dataset (including CNN / Daily Mail), it yields very good results.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = TFAutoModelWithLMHead.from_pretrained("t5-base")

# T5 uses a max_length of 512 so we cut the article to 512 tokens.
inputs = tokenizer.encode("summarize: " + ARTICLE, return_tensors="tf", max_length=512)
outputs = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

In [8]:
print(outputs)

tf.Tensor(
[[    0     3 29905   497     8  5281     7   130   294    13    46 10653
  13236     3     5     3    99     3 21217     6  1207  3483   235     7
   8519   192  4336 12052    13    96  1647    49    53     3     9  6136
   5009    21  9479    16     8   166  1952   121   255    65   118  4464
    335   648     6  4169    13   135   344  5247    11  4407     3     5]], shape=(1, 60), dtype=int32)


In [12]:
print(tokenizer.decode(outputs[0]))

<pad> prosecutors say the marriages were part of an immigration scam. if convicted, barrientos faces two criminal counts of "offering a false instrument for filing in the first degree" she has been married 10 times, nine of them between 1999 and 2002.


## PyTorch implementation

In [ ]:
# Pytorch
tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = AutoModelWithLMHead.from_pretrained("t5-base")

# T5 uses a max_length of 512 so we cut the article to 512 tokens.
inputs = tokenizer.encode("summarize: " + ARTICLE, return_tensors="pt", max_length=512)
outputs = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

In [15]:
print(outputs)

tensor([[    0,     3, 29905,   497,     8,  5281,     7,   130,   294,    13,
            46, 10653, 13236,     3,     5,     3,    99,     3, 21217,     6,
          1207,  3483,   235,     7,  8519,   192,  4336, 12052,    13,    96,
          1647,    49,    53,     3,     9,  6136,  5009,    21,  9479,    16,
             8,   166,  1952,   121,   255,    65,   118,  4464,   335,   648,
             6,  4169,    13,   135,   344,  5247,    11,  4407,     3,     5,
             1]])


In [16]:
print(tokenizer.decode(outputs[0]))

<pad> prosecutors say the marriages were part of an immigration scam. if convicted, barrientos faces two criminal counts of "offering a false instrument for filing in the first degree" she has been married 10 times, nine of them between 1999 and 2002.</s>
